In [ ]:
#Planning for employees
#The staff planning representes a list per day
#Each day consists of many lists of length 3
#Each list of 3 is an employee with the following items: staff id, starting time, shift duration
#The number of lists is the number of casts that are possibly available on that day

In [19]:
import numpy as np
import pandas as pd

In [7]:
staff_planning = []
for k in range(4):
    list = []
    cnt = 0
    for i in range(11):

        temp = []
        for j in range(3):       
            if j == 0:
                temp.append(cnt)
                cnt = cnt + 1
            elif j == 1:
                temp.append(0)
            else:
                temp.append(10)
        list.append(temp)
    staff_planning.append(list)


In [8]:
staff_planning

[[[0, 0, 10],
  [1, 0, 10],
  [2, 0, 10],
  [3, 0, 10],
  [4, 0, 10],
  [5, 0, 10],
  [6, 0, 10],
  [7, 0, 10],
  [8, 0, 10],
  [9, 0, 10],
  [10, 0, 10]],
 [[0, 0, 10],
  [1, 0, 10],
  [2, 0, 10],
  [3, 0, 10],
  [4, 0, 10],
  [5, 0, 10],
  [6, 0, 10],
  [7, 0, 10],
  [8, 0, 10],
  [9, 0, 10],
  [10, 0, 10]],
 [[0, 0, 10],
  [1, 0, 10],
  [2, 0, 10],
  [3, 0, 10],
  [4, 0, 10],
  [5, 0, 10],
  [6, 0, 10],
  [7, 0, 10],
  [8, 0, 10],
  [9, 0, 10],
  [10, 0, 10]],
 [[0, 0, 10],
  [1, 0, 10],
  [2, 0, 10],
  [3, 0, 10],
  [4, 0, 10],
  [5, 0, 10],
  [6, 0, 10],
  [7, 0, 10],
  [8, 0, 10],
  [9, 0, 10],
  [10, 0, 10]]]

In [ ]:
#Planning for shop
#Based on previous day we know how many staff we need each hour
#The staff needed is in the following shape
    #-list of days
    #-each dya is a list of 24 hours, with the number of employees

In [12]:
hourlystaff_planning = []
for i in range(5):
    list = []
    for j in range(24):
        if j in range(0,7):
            list.append(0)
        if j in range(7, 10):
            list.append(4)
        if j in range(10, 13):
            list.append(2)
        if j in range(13, 15):
            list.append(6)
        if j in range(15,18):
            list.append(2)
        if j in range(18,21):
            list.append(6)
        if j in range(21, 25):
            list.append(2)
    hourlystaff_planning.append(list)
                        

In [17]:
hourlystaff_planning

array([[0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2,
        2, 2],
       [0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2,
        2, 2],
       [0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2,
        2, 2],
       [0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2,
        2, 2],
       [0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2,
        2, 2]])

In [16]:
hourlystaff_planning = np.array(hourlystaff_planning)

In [ ]:
#the cost function is calculated as hours understaffed + hours overstaffed

def cost(hourlystaff, hourlystaff_needed):
    errors = hourlystaff - hourlystaff_needed
    overstaff = abs(errors[errors > 0].sum())
    understaff = abs(errors[errors < 0].sum())
    
    overstaff_cost = 1
    understaff_cost = 1
    
    cost = overstaff_cost * overstaff + understaff_cost * understaff
    
    return cost

There are two key steps in the Genetic Algorithm: mating (also cross-over or recombination) and mutation.
In the Mating step, the new generation is formed from the offspring of individuals of the parent population, as it is in natural selection.
To apply this to our example, consider that later on, we will be generating many not so good staff plannings and trying to combine the best ones together. So we need to define a way to “mix” two individuals (staff plannings) with each other.
In this example, I have decided to code this as follows:
* Choose a random mom from the population
* Choose a random dad from the population
* Create a child as the same size as a parent, but randomly filled with zeroes and ones.
* The positions where the child has ones, we take the data from his father, and the positions where the child has zeroes, we take the data from his mother.
* We repeat this for each child (the number of children is equal to the population size)

In [18]:
# for each iteration select randomly two parents and make a random combination of those two parents
# by applying a randomly generated yes/no mask to the two selected parents
#Cross-Over for the Genetic Algorithm 
def random_combine(parents, n_offspring):
    n_parents = len(parents)
    n_periods = len(parents[0])
    n_employees = len(parents[0][0])
    
    offspring = []
    
    for i in range(n_offspring):
        random_dad = parents[np.random.randint(low = 0, high = n_parents - 1)]
        random_mom = parents[np.random.randint(low = 0, high = n_parents - 1)]
        
        dad_mask = np.random.randint(0,2, size = np.array(random_dad).shape)
        mom_mask= np.logical_not(dad_mask)
        
        child = np.add(np.multiply(random_dad, dad_mask), np.multiply(random_mom, random_mask))
        
        offspring.append(child)
        
    return offspring

In [ ]:
def mutate_parent(parent, n_mutations):
    size1 = parent.shape[0] #number of periods
    size2 = parent.shape[1] #number of employees
    
    for i in range(n_mutations):
        rand1 = np.random.randint(0,size1)
        rand2 = np.random.randint(0,size2)
        rand3 = np.random.randint(1,2)
        
        parent[rand1, rand2, rand3] = np.random.randint(0,10)
    return parent

def mutate_gen(parent_gen, n_mutations):
    mutated_parent_gen = []
    
    for parent in parent_gen:
        mutated_parent_gen.append(mutate_parent(parent, n_mutations))
    
    return mutated_parent_gen

In [ ]:
#Select all feasible solutions: remove those in which employees work more than 10 hours
def is_acceptable(parent):
    return np.logical_not((np.array(parent)[:,:,2] > 10).any())

def selected_acceptable(parent_gen):
    parent_gen = [parent for parent in parent_gen if is_acceptable(parent)]
    return parent_gen

In [22]:
def select_best(parent_gen, hourlystaff_needed, n_best):
    costs = []
    
    for idx, parent_staff_planning in enumerate(parent_gen):
        parent_hourly_planning = staffplanning_to_hourlyplanning(parent_staff_planning)
        parent_cost = cost(parent_hourly_planning, hourlystaff_needed)
        costs.append([idx, parent_cost])
        
    costs_temp = pd.DataFrame(costs).sort_values(by = 1, ascending = True).reset_index(drop = True)
    selected_parents_idx = list(costs_tmp.iloc[:n_best, 0])
    selected_parents = [parent for idx, parent in enumerate(parent_gen) if idx in selected_parents_idx]
    
    return selected_parents

In [24]:
#Overall function

def gen_algo(hourlystaff_needed, n_iterations):
    generation_size = 500
    
    parent_gen = create_parent_generation(n_parents = generation_size, n_days = 5, n_staff = 11)
    for it in range(n_iterations):
        parent_gen = select_acceptable(parent_gen)
        parent_gen = select_best(parent_gen, hourlystaff_needed, n_best = 100)
        parent_gen = random_combine(parent_gen, n_offspring = generation_size)
        parent_gen = mutate_gen(parent_gen, n_mutations = 1)
        
    best_child = select_best(parent_gen, hourlystaff_needed, n_best = 1)
    
    return best_child